In [117]:
import numpy as np
import time
import os

In [118]:
current_path = os.getcwd()
filename = os.path.join(current_path, "matrix_100x100.csv")
matrix1 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix1),np.array_equal(matrix1, matrix1.T))

filename = os.path.join(current_path, "matrix_200x200.csv")
matrix2 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix2),np.array_equal(matrix2, matrix2.T))

filename = os.path.join(current_path, "matrix_400x400.csv")
matrix4 = np.genfromtxt(filename, delimiter=',').astype(np.float32)
print(np.shape(matrix4),np.array_equal(matrix4, matrix4.T))

filename = os.path.join(current_path, "matrix_800x800.csv")
matrix8 = np.genfromtxt(filename, delimiter=',',).astype(np.float32)
print(np.shape(matrix8),np.array_equal(matrix8, matrix8.T))

(100, 100) True
(200, 200) True
(400, 400) True
(800, 800) True


In [119]:
def norm(v):
    return np.sqrt(np.dot(v,v))

def normalize(v):
    return v/norm(v)

In [120]:
def qr_householder(A):
    m, n = A.shape
    R = A.astype(float)  # Avoid unnecessary copy, ensure mutability
    Q = np.eye(m)

    for i in range(n - 1):
        # Compute the Householder vector
        u = R[i:, i].copy()  # Work with a slice; make a copy to modify
        alpha = norm(u)
        u[0] -= alpha  # Create the Householder vector in place
        norm_u = norm(u)

        if norm_u != 0:  # Avoid division by zero
            u /= norm_u

            # Apply the reflection to R in place
            R[i:, i:] -= 2 * np.outer(u, u @ R[i:, i:])
            
            # Apply the reflection to Q in place
            Q[:, i:] -= 2 * np.outer(Q[:, i:] @ u, u)
    return Q, R


In [121]:
def createRQ(A):
    Q, R = qr_householder(A)
    B = np.matmul(R, Q)
    return B,Q

In [122]:
epsilon = 1e-8  # Using exponentiation

In [123]:
def qr_algorithm_with_shifts(A, refeig=-1, sindex=-1):
    # Compute the initial QR decomposition of A
    B, Q = createRQ(A)
    
    n = 0  # Iteration counter
    leig = B[refeig, refeig]  # Initial reference eigenvalue
    diff = 1  # Difference tracker for convergence

    # Iterate until the eigenvalues stabilize (very small difference)
    while diff > epsilon:
        shift = np.eye(len(B)) * B[sindex, sindex]  # Create shift matrix
        C = B - shift  # Apply shift to B
        B, Q = createRQ(C)  # Perform QR decomposition
        B = B + shift  # Undo the shift

        n += 1  # Count iterations
        diff = abs(leig - B[refeig, refeig])  # Compute change in eigenvalue
        leig = B[refeig, refeig]  # Update reference eigenvalue

    # Extract eigenvalues (diagonal elements of B)
    eigs = [B[i, i] for i in range(len(B))]
    
    # Extract eigenvectors (columns of Q)
    eigv = [Q[:, i] for i in range(Q.shape[1])]
    return eigs, eigv



In [124]:
matrix = matrix8
number_of_eigenvectors_for_print = 5

In [125]:
# Measure time for qr_shifts
start_time = time.time()
eigs_qr, eigv_qr = qr_algorithm_with_shifts(matrix, refeig=-1, sindex=-1)
qr_time = time.time() - start_time
print(f"\nExecution time for qr_shifts: {qr_time:.6f} seconds")

# Measure time for NumPy's eig function
start_time = time.time()
eigs_np, eigv_np = np.linalg.eig(matrix)
np_time = time.time() - start_time
print(f"Execution time for np.linalg.eig: {np_time:.6f} seconds")


Execution time for qr_shifts: 12.297300 seconds
Execution time for np.linalg.eig: 0.638328 seconds


In [127]:
# Sort and compare the results
eigs_qr_sorted = sorted(eigs_qr)
eigs_np_sorted = sorted(eigs_np)

print("\nEigenvalues from qr_shifts:")
print(eigs_qr_sorted)

print("\nEigenvalues from np.linalg.eig:")
print(eigs_np_sorted)

# Check the maximum difference between computed eigenvalues
max_diff = max(abs(np.array(eigs_qr_sorted) - np.array(eigs_np_sorted)))
print(f"\nMax difference between methods: {max_diff}")



Eigenvalues from qr_shifts:
[np.float64(-2.6228162089762765), np.float64(-2.248675144918251), np.float64(-1.549549973052186), np.float64(-1.537825372209126), np.float64(-1.4596847111615772), np.float64(-1.3956963932223756), np.float64(-1.3909246507693234), np.float64(-1.3863557386989835), np.float64(-1.3742133150495244), np.float64(-1.364513962392597), np.float64(-1.337160316088027), np.float64(-1.3269187290988904), np.float64(-1.3110305199487378), np.float64(-1.2521329835595678), np.float64(-1.2277630460710116), np.float64(-1.20925103585469), np.float64(-1.206949390876406), np.float64(-1.203992693600343), np.float64(-1.1561427667390474), np.float64(-1.1525067388181658), np.float64(-1.1422857610675026), np.float64(-1.1313804816187827), np.float64(-1.0805934740759584), np.float64(-1.054856610509153), np.float64(-1.0515660685937007), np.float64(-1.0372886236291865), np.float64(-1.0233390290406115), np.float64(-1.0069518759464673), np.float64(-0.9841590970369168), np.float64(-0.983321337

In [128]:
# Print eigenvectors
print(f"\nFirst {number_of_eigenvectors_for_print} eigenvectors from qr_shifts:")
for i in range(number_of_eigenvectors_for_print):
    print(eigv_qr[i])

print(f"\nFirst {number_of_eigenvectors_for_print} eigenvectors from np.linalg.eig:")
for i in range(number_of_eigenvectors_for_print):
    print(eigv_np[:, i])


First 5 eigenvectors from qr_shifts:
[ 9.99965606e-01  5.91388595e-03  3.32832202e-03  2.37326041e-03
  1.95328162e-03  1.58651414e-03  1.32455406e-03  7.29947136e-04
  8.68368655e-04  1.19701792e-03  1.01902112e-03  5.36672211e-04
  7.52071012e-04  5.17094673e-04  4.61322933e-04  5.93586667e-04
  3.55354710e-04  2.44061921e-04  4.14478037e-04  3.99398566e-04
  3.65942356e-04  2.82780015e-04  3.71635622e-04  3.69350228e-04
  3.04835231e-04  3.74771653e-04  4.14052185e-04  2.29416187e-04
  4.07299178e-04  1.50111523e-04  3.29336393e-04  1.98043744e-04
  7.75180609e-05  3.46146290e-04  3.07564833e-04  1.67719856e-04
  2.24642503e-04  2.53318489e-04  1.55988045e-04  3.17428000e-04
  1.78383358e-04  1.07433627e-04  1.68460811e-04  1.82395161e-04
  2.37030254e-04  1.07248898e-04  1.24515750e-04  1.95242132e-04
  1.78155447e-04 -5.38537220e-06  1.09355250e-04  1.38412325e-04
  6.86984506e-05  1.18920681e-04  1.38145235e-04  6.32191046e-05
  2.04785279e-04  8.52787195e-05  1.52282136e-04  2.